Importation des library

In [1]:
import numpy as np
import pandas as pd
import vectorbt as vbt
from itertools import product
# ============================================================
# CONFIGURATION DATASETS
# ============================================================
pd.set_option('future.no_silent_downcasting', True)


DATASETS = {
    "ETHUSDT_1h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\1h\ETHUSDT\ETHUSDT_3years_2022-12-22_TO_2025-12-26_1h_data",
}

INIT_CASH = 1000
FEES = 0.001

# === CONFIG STRATÉGIE ===
RSI_LENGTH = 14
LOOKBACK_LEFT = 3
LOOKBACK_RIGHT = 3

ATR_LENGTH = 14
ATR_MULT = 3.0

MIN_DIV_BARS = 10
MAX_DIV_BARS = 200

TRAIN_RATIO = 0.6
TEST_RATIO = 0.2

# ============================================================
# STRATÉGIE
# ============================================================

def run_strategy_atr(df):
    high = df["High"].astype(float)
    low = df["Low"].astype(float)
    close = df["Close"].astype(float).ffill().infer_objects(copy=False)

    rsi = vbt.RSI.run(close, window=RSI_LENGTH).rsi
    atr = vbt.ATR.run(high, low, close, window=ATR_LENGTH).atr

    window = LOOKBACK_LEFT + LOOKBACK_RIGHT + 1

    pivot_low_price = low.shift(LOOKBACK_RIGHT) == low.rolling(window).min()
    pivot_low_rsi = rsi.shift(LOOKBACK_RIGHT) == rsi.rolling(window).min()

    bullish_div = pd.Series(False, index=df.index)

    idx = np.where(pivot_low_price & pivot_low_rsi)[0]

    if len(idx) >= 2:
        idx1 = idx[1:]
        idx2 = idx[:-1]
        bars = idx1 - idx2

        valid = (
            (low.iloc[idx1].values < low.iloc[idx2].values) &
            (rsi.iloc[idx1].values > rsi.iloc[idx2].values) &
            (bars >= MIN_DIV_BARS) &
            (bars <= MAX_DIV_BARS)
        )

        bullish_div.iloc[idx1[valid] - LOOKBACK_RIGHT] = True

    entries = bullish_div.shift(1).fillna(False)

    # ========================================================
    # ATR TRAILING STOP (manuel)
    # ========================================================

    in_position = False
    entry_price = 0.0
    trailing_stop = np.nan

    exits = pd.Series(False, index=df.index)
    stop_series = pd.Series(np.nan, index=df.index)

    for i in range(len(df)):
        if entries.iloc[i] and not in_position:
            in_position = True
            entry_price = close.iloc[i]
            trailing_stop = entry_price - ATR_MULT * atr.iloc[i]

        elif in_position:
            trailing_stop = max(
                trailing_stop,
                close.iloc[i] - ATR_MULT * atr.iloc[i]
            )

            if close.iloc[i] <= trailing_stop:
                exits.iloc[i] = True
                in_position = False
                trailing_stop = np.nan

        stop_series.iloc[i] = trailing_stop


    entries = bullish_div.shift(1).fillna(False).astype(bool)
    exits = exits.astype(bool)
    close_arr = close.to_numpy(dtype=float)


    pf = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        exits=exits,
        init_cash=INIT_CASH,
        fees=FEES,
        direction="longonly"
    )

    return pf

# ============================================================
# WALK FORWARD
# ============================================================

def walk_forward(df):
    n = len(df)
    train_size = int(n * TRAIN_RATIO)
    test_size = int(n * TEST_RATIO)

    results = []
    start = 0
    step = test_size
    fold = 1

    while start + train_size + test_size <= n:
        test_df = df.iloc[start + train_size:start + train_size + test_size]

        pf = run_strategy_atr(test_df)
        stats = pf.stats()

        results.append({
            "fold": fold,
            "start": test_df.index[0],
            "end": test_df.index[-1],
            "return": stats["Total Return [%]"],
            "drawdown": stats["Max Drawdown [%]"],
            "trades": pf.trades.count()
        })

        fold += 1
        start += step

    return pd.DataFrame(results)

# ============================================================
# RUN
# ============================================================

if __name__ == "__main__":
    for name, path in DATASETS.items():
        print(f"\n▶ Walk-forward ATR trailing : {name}")
        df = pd.read_parquet(path)

        wf = walk_forward(df)
        wf["dataset"] = name

        print(wf)
        print("\nMoyenne :\n", wf[["return", "drawdown", "trades"]].mean())


▶ Walk-forward ATR trailing : ETHUSDT_1h
   fold                     start                       end     return  \
0     1 2024-10-12 11:00:00+01:00 2025-05-20 13:00:00+01:00  -0.965532   
1     2 2025-05-20 14:00:00+01:00 2025-12-26 16:00:00+01:00  31.532840   

    drawdown  trades     dataset  
0  10.688158      14  ETHUSDT_1h  
1  15.536793      25  ETHUSDT_1h  

Moyenne :
 return      15.283654
drawdown    13.112475
trades      19.500000
dtype: float64
